In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import keras
import cv2
from tensorflow.keras import datasets,layers,models
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle

In [2]:
X_train=np.load("BOTTOM_TRAIN.npy")
y_train=np.load("y_final_train.npy")
X_cval=np.load("X_final_cval.npy")
Y_cval=np.load("y_final_cval.npy")

In [3]:
X_train.shape

(38400, 128, 128, 3)

In [4]:
data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.experimental.preprocessing.RandomContrast(0.7),
    
])

In [5]:
cnn_bottom= models.Sequential([
    data_augmentation,
    # cnn
    layers.Conv2D(filters = 32,kernel_size = (3,3),activation = 'relu',input_shape = (128,128,3),padding = 'same'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(filters = 32,kernel_size = (3,3),activation = 'relu',input_shape = (128,128,3),padding = 'same'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(filters = 32,kernel_size = (3,3),activation = 'relu',input_shape = (128,128,3),padding = 'same'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(filters = 32,kernel_size = (3,3),activation = 'relu',input_shape = (128,128,3),padding = 'same'),
    layers.MaxPooling2D((2,2)),
    
    #layers.Conv2D(filters = 32,kernel_size = (3,3),activation = 'relu',input_shape = (128,128,3),padding = 'same'),
    #layers.MaxPooling2D((2,2)),
    
    #layers.Conv2D(filters = 32,kernel_size = (3,3),activation = 'relu',input_shape = (128,128,3),padding = 'same'),
    #layers.MaxPooling2D((2,2)),
    
    layers.Dropout(0.2),
    
    # dense
    layers.Flatten(),
    layers.Dense(64,activation = "relu"),
    #layers.Dense(64,activation = "relu"),
    layers.Dense(16,activation = 'softmax')
]
)

In [6]:
cnn_bottom.compile(
    optimizer='adam',
    loss="sparse_categorical_crossentropy",
    metrics="accuracy"   
)

In [7]:
cnn_bottom.fit(X_train,y_train,epochs=50, validation_data=(X_cval,Y_cval))

Epoch 1/50
1200/1200 [==============================] - 683s 565ms/step - loss: 2.3653 - accuracy: 0.2379 - val_loss: 1.9760 - val_accuracy: 0.3808
Epoch 2/50
1200/1200 [==============================] - 656s 547ms/step - loss: 1.9429 - accuracy: 0.3887 - val_loss: 1.7921 - val_accuracy: 0.4445
Epoch 3/50
1200/1200 [==============================] - 641s 534ms/step - loss: 1.7893 - accuracy: 0.4418 - val_loss: 1.6949 - val_accuracy: 0.4719
Epoch 4/50
1200/1200 [==============================] - 652s 543ms/step - loss: 1.6759 - accuracy: 0.4750 - val_loss: 1.5636 - val_accuracy: 0.5210
Epoch 5/50
1200/1200 [==============================] - 647s 539ms/step - loss: 1.5840 - accuracy: 0.5063 - val_loss: 1.5633 - val_accuracy: 0.5178
Epoch 6/50
1200/1200 [==============================] - 649s 540ms/step - loss: 1.5087 - accuracy: 0.5314 - val_loss: 1.4961 - val_accuracy: 0.5454
Epoch 7/50
1200/1200 [==============================] - 644s 536ms/step - loss: 1.4542 - accuracy: 0.5500 - val_

In [8]:
cnn_bottom.save("BOTTOM_MODEL.h5")

In [13]:
new_path="C:/Users/nithi/.jupyter/new_validation"

In [14]:
data = range(17801,18701)
df = pd.DataFrame(data,columns = ['id'])

In [15]:
X_test = []
for i in range(len(df)):
    path = os.path.join(new_path,str(df.iloc[i,0]))
    path = path.replace("\\","/")
    path = path+".tif"
    path=path.replace("\new","/new")
    #path = os.path.join(validation_dir,str(df.iloc[i,0]))
    #path = path.replace("\\","/")
    #path = path + ".tif"
    img = cv2.imread(path)
    img = img / 255
    resized_img = cv2.resize(img,(128,128))
    X_test.append(resized_img)

In [17]:
X_test = np.array(X_test)
predictions=cnn_bottom.predict(X_test)

29/29 [==============================] - 4s 124ms/step


In [18]:
predictions[:10]

array([[4.41479131e-07, 6.06451522e-06, 4.63988698e-07, 9.94566653e-06,
        9.98976111e-01, 1.78650458e-04, 9.14865177e-06, 1.20948630e-06,
        7.74101045e-06, 3.76106997e-04, 3.45834962e-07, 5.30073692e-07,
        8.60242799e-05, 3.30621115e-05, 3.13697150e-04, 4.13588225e-07],
       [7.12279825e-06, 9.97572101e-07, 2.69580619e-06, 5.68534331e-09,
        1.95145622e-05, 5.41648711e-04, 9.87799525e-01, 2.74716232e-08,
        1.70008832e-07, 1.15956916e-02, 1.23305298e-07, 3.33036515e-10,
        3.91636104e-06, 3.04062610e-06, 2.54823390e-05, 3.72710538e-08],
       [1.03703826e-07, 2.95038581e-06, 4.21395940e-09, 3.78113909e-05,
        9.82132554e-01, 8.85669215e-06, 6.60087899e-05, 4.94292035e-05,
        4.01923637e-04, 2.85868370e-03, 1.42269897e-07, 1.31691445e-06,
        1.89290047e-04, 1.42490910e-02, 1.66141785e-06, 1.78015725e-07],
       [7.21245212e-03, 1.92024279e-02, 4.37832532e-05, 2.24211179e-02,
        2.69853114e-03, 2.60220398e-03, 1.31284760e-03, 2.410

In [19]:
y_classes_1 = [np.argmax(element) for element in predictions]
y_classes_1[:10]

[4, 6, 4, 8, 3, 8, 11, 3, 2, 4]

In [20]:
df3 = pd.DataFrame(y_classes_1,columns = ['label'])
result_1 = pd.concat([df, df3], axis=1)
result_1.set_index('id',inplace = True)
result_1.to_csv("C:/Users/nithi/.jupyter/bottom_results.csv")